# Assignment 9: GBDT

#### Response Coding: Example

<img src='http://i.imgur.com/TufZptV.jpg' width=700px>

> The response tabel is built only on train dataset.
> For a category which is not there in train data and present in test data, we will encode them with default values
Ex: in our test data if have State: D then we encode it as [0.5, 0.05]

<ol>
    <li><strong>Apply GBDT on these feature sets</strong>
        <ul>
            <li><font color='red'>Set 1</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF)+  preprocessed_eassay (TFIDF)+sentiment Score of eassay(check the bellow example, include all 4 values as 4 features)</li>
            <li><font color='red'>Set 2</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF W2V)+  preprocessed_eassay (TFIDF W2V)</li>        </ul>
    </li>
    <li><strong>The hyper paramter tuning (Consider any two hyper parameters)</strong>
        <ul>
    <li>Find the best hyper parameter which will give the maximum <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/receiver-operating-characteristic-curve-roc-curve-and-auc-1/'>AUC</a> value</li>
    <li>find the best hyper paramter using k-fold cross validation/simple cross validation data</li>
    <li>use gridsearch cv or randomsearch cv or you can write your own for loops to do this task</li>
        </ul>
    </li>
    <li>
    <strong>Representation of results</strong>
        <ul>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/Gp2DQmh.jpg' width=500px> with X-axis as <strong>n_estimators</strong>, Y-axis as <strong>max_depth</strong>, and Z-axis as <strong>AUC Score</strong> , we have given the notebook which explains how to plot this 3d plot, you can find it in the same drive <i>3d_scatter_plot.ipynb</i></li>
            <p style="text-align:center;font-size:30px;color:red;"><strong>or</strong></p> <br>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/fgN9aUP.jpg' width=300px> <a href='https://seaborn.pydata.org/generated/seaborn.heatmap.html'>seaborn heat maps</a> with rows as <strong>n_estimators</strong>, columns as <strong>max_depth</strong>, and values inside the cell representing <strong>AUC Score</strong> </li>
    <li>You choose either of the plotting techniques out of 3d plot or heat map</li>
    <li>Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    <img src='https://i.imgur.com/wMQDTFe.jpg' width=300px></li>
    <li>Along with plotting ROC curve, you need to print the <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/confusion-matrix-tpr-fpr-fnr-tnr-1/'>confusion matrix</a> with predicted and original labels of test data points
    <img src='https://i.imgur.com/IdN5Ctv.png' width=300px></li>
            </ul>
    <br>
    <li>You need to summarize the results at the end of the notebook, summarize it in the table format
        <img src='http://i.imgur.com/YVpIGGE.jpg' width=400px>
    </li>
</ol>

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# import nltk
# nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

for_sentiment = 'a person is a person no matter how small dr seuss i teach the smallest students with the biggest enthusiasm \
for learning my students learn in many different ways using all of our senses and multiple intelligences i use a wide range\
of techniques to help all my students succeed students in my class come from a variety of different backgrounds which makes\
for wonderful sharing of experiences and cultures including native americans our school is a caring community of successful \
learners which can be seen through collaborative student project based learning in and out of the classroom kindergarteners \
in my class love to work with hands on materials and have many different opportunities to practice a skill before it is\
mastered having the social skills to work cooperatively with friends is a crucial aspect of the kindergarten curriculum\
montana is the perfect place to learn about agriculture and nutrition my students love to role play in our pretend kitchen\
in the early childhood classroom i have had several kids ask me can we try cooking with real food i will take their idea \
and create common core cooking lessons where we learn important math and writing concepts while cooking delicious healthy \
food for snack time my students will have a grounded appreciation for the work that went into making the food and knowledge \
of where the ingredients came from as well as how it is healthy for their bodies this project would expand our learning of \
nutrition and agricultural cooking recipes by having us peel our own apples to make homemade applesauce make our own bread \
and mix up healthy plants from our classroom garden in the spring we will also create our own cookbooks to be printed and \
shared with families students will gain math and literature skills as well as a life long enjoyment for healthy cooking \
nannan'
ss = sid.polarity_scores(for_sentiment)

for k in ss:
    print('{0}: {1}, '.format(k, ss[k]), end='')

# we can use these 4 things as features/attributes (neg, neu, pos, compound)
# neg: 0.0, neu: 0.753, pos: 0.247, compound: 0.93

<h1>1. GBDT (xgboost/lightgbm) </h1>

## 1.1 Loading Data

In [ ]:

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import MultinomialNB


import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/

import pickle
from tqdm import tqdm
import os

#from plotly import plotly
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode()
from collections import Counter

In [ ]:
data = pd.read_csv('C:/Users/Abhishek V/Downloads/preprocessed_data.csv')
data.head()

In [ ]:

y = data['project_is_approved'].values
X = data.drop(['project_is_approved'], axis=1)
X.head(1)

<h2>1.2 Splitting data into Train and cross validation(or test): Stratified Sampling</h2>

In [ ]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

In [ ]:

#1.2.1 Splitting data into Train and cross validation(or test): Stratified Sampling [ ]
# train test split
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.33, stratify=y)
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33, stratify=y_train)

<h2>1.3 Make Data Model Ready: encoding eassay, and project_title</h2>

In [ ]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

In [ ]:

print(X_train.shape, y_train.shape)
print(X_cv.shape, y_cv.shape)
print(X_test.shape, y_test.shape)

print("="*100)


vectorizer = TfidfVectorizer(min_df=10,ngram_range=(1,4), max_features=5000)

# encoding eassay
vectorizer.fit(X_train['essay'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_essay_tfidf = vectorizer.transform(X_train['essay'].values)
X_cv_essay_tfidf = vectorizer.transform(X_cv['essay'].values)
X_test_essay_tfidf = vectorizer.transform(X_test['essay'].values)

print("After vectorizations")
print(X_train_essay_tfidf.shape, y_train.shape)
print(X_cv_essay_tfidf.shape, y_cv.shape)
print(X_test_essay_tfidf.shape, y_test.shape)
print("="*100)

<h2>1.4 Make Data Model Ready: encoding numerical, categorical features</h2>

In [ ]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding 
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

In [ ]:
def get_gv_fea_dict(alpha, feature, df):
    
    value_count = X_train[feature].value_counts()
    
    # gv_dict : Gene Variation Dict, which contains the probability array for each gene/variation
    gv_dict = dict()
    
    # denominator will contain the number of time that particular feature occured in whole data
    for i, denominator in value_count.items():
        # vec will contain (p(yi==1/Gi) probability of gene/variation belongs to perticular class
        # vec is 9 diamensional vector
        vec = []
        for k in range(0,2):
            # print(train_df.loc[(train_df['Class']==1) & (train_df['Gene']=='BRCA1')])
            
            # cls_cnt.shape[0] will return the number of rows

            cls_cnt = X_train.loc[(X_train['project_is_approved']==k) & (X_train[feature]==i)]
            
            # cls_cnt.shape[0](numerator) will contain the number of time that particular feature occured in whole data
            vec.append((cls_cnt.shape[0] + alpha*10)/ (denominator + 90*alpha))

        # we are adding the gene/variation to the dict as key and vec as value
        gv_dict[i]=vec
    return gv_dict

# Get Gene variation feature
def get_gv_feature(alpha, feature, df):
    # print(gv_dict)
    
    gv_dict = get_gv_fea_dict(alpha, feature, df)
    # value_count is similar in get_gv_fea_dict
    value_count = X_train[feature].value_counts()
    
    # gv_fea: Gene_variation feature, it will contain the feature for each feature value in the data
    gv_fea = []
    # for every feature values in the given data frame we will check if it is there in the train data then we will add the feature to gv_fea
    # if not we will add [1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9,1/9] to gv_fea
    for index, row in df.iterrows():
        if row[feature] in dict(value_count).keys():
            gv_fea.append(gv_dict[row[feature]])
        else:
            gv_fea.append([.5,.5])
#             gv_fea.append([-1,-1,-1,-1,-1,-1,-1,-1,-1])
    return gv_fea


## 1.4.1 Response encoding categorical features: School State

In [ ]:
#response-coding of the School State feature
# alpha is used for laplace smoothing
alpha = 1
# train state feature
X_train_state_feature_responseCoding = np.array(get_gv_feature(alpha, "school_state", X_train))
# test state feature
X_test_state_feature_responseCoding = np.array(get_gv_feature(alpha, "school_state", X_test))
# cross validation state feature
X_cv_state_feature_responseCoding = np.array(get_gv_feature(alpha, "school_state", X_cv))

print("After vectorizations")
print(X_train_state_feature_responseCoding.shape, y_train.shape)
print(X_cv_state_feature_responseCoding.shape, y_cv.shape)
print(X_test_state_feature_responseCoding.shape, y_test.shape)
#print(vectorizer.get_feature_names())
print("="*100)


## 1.4.2 Response encoding categorical features: teacher_prefix

In [ ]:
#response-coding of the teacher_prefix feature
# alpha is used for laplace smoothing
alpha = 1
# train state feature
X_train_teacher_prefix_feature_responseCoding = np.array(get_gv_feature(alpha, "teacher_prefix", X_train))
# test state feature
X_test_teacher_prefix_feature_responseCoding = np.array(get_gv_feature(alpha, "teacher_prefix", X_test))
# cross validation state feature
X_cv_teacher_prefix_feature_responseCoding = np.array(get_gv_feature(alpha, "teacher_prefix", X_cv))

print("After vectorizations")
print(X_train_teacher_prefix_feature_responseCoding.shape, y_train.shape)
print(X_cv_teacher_prefix_feature_responseCoding.shape, y_cv.shape)
print(X_test_teacher_prefix_feature_responseCoding.shape, y_test.shape)
#print(vectorizer.get_feature_names())
print("="*100)


## 1.4.3 Response encoding categorical features: project_grade_category

In [ ]:
#response-coding of the project_grade_category feature
# alpha is used for laplace smoothing
alpha = 1
# train state feature
X_train_project_grade_category_feature_responseCoding = np.array(get_gv_feature(alpha, "project_grade_category", X_train))
# test state feature
X_test_project_grade_category_feature_responseCoding = np.array(get_gv_feature(alpha, "project_grade_category", X_test))
# cross validation state feature
X_cv_project_grade_category_feature_responseCoding = np.array(get_gv_feature(alpha, "project_grade_category", X_cv))

print("After vectorizations")
print(X_train_project_grade_category_feature_responseCoding.shape, y_train.shape)
print(X_cv_project_grade_category_feature_responseCoding.shape, y_cv.shape)
print(X_test_project_grade_category_feature_responseCoding.shape, y_test.shape)
#print(vectorizer.get_feature_names())
print("="*100)

## 1.4.4 Response encoding categorical features: clean_categories

In [ ]:

#response-coding of the clean_categories feature
# alpha is used for laplace smoothing
alpha = 1
# train state feature
X_train_clean_categories_feature_responseCoding = np.array(get_gv_feature(alpha, "clean_categories", X_train))
# test state feature
X_test_clean_categories_feature_responseCoding = np.array(get_gv_feature(alpha, "clean_categories", X_test))
# cross validation state feature
X_cv_clean_categories_feature_responseCoding = np.array(get_gv_feature(alpha, "clean_categories", X_cv))

print("After vectorizations")
print(X_train_clean_categories_feature_responseCoding.shape, y_train.shape)
print(X_cv_clean_categories_feature_responseCoding.shape, y_cv.shape)
print(X_test_clean_categories_feature_responseCoding.shape, y_test.shape)
#print(vectorizer.get_feature_names())
print("="*100)

## 1.4.5 Response encoding categorical features: clean_subcategories

In [ ]:

#response-coding of the clean_subcategories feature
# alpha is used for laplace smoothing
alpha = 1
# train state feature
X_train_clean_subcategories_feature_responseCoding = np.array(get_gv_feature(alpha, "clean_subcategories", X_train))
# test state feature
X_test_clean_subcategories_feature_responseCoding = np.array(get_gv_feature(alpha, "clean_subcategories", X_test))
# cross validation state feature
X_cv_clean_subcategories_feature_responseCoding = np.array(get_gv_feature(alpha, "clean_subcategories", X_cv))

print("After vectorizations")
print(X_train_clean_subcategories_feature_responseCoding.shape, y_train.shape)
print(X_cv_clean_subcategories_feature_responseCoding.shape, y_cv.shape)
print(X_test_clean_subcategories_feature_responseCoding.shape, y_test.shape)
#print(vectorizer.get_feature_names())
print("="*100)


## 1.4.6 encoding numerical features: Price

In [ ]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
# normalizer.fit(X_train['price'].values)
# this will rise an error Expected 2D array, got 1D array instead: 
# array=[105.22 215.96  96.01 ... 368.98  80.53 709.67].
# Reshape your data either using 
# array.reshape(-1, 1) if your data has a single feature 
# array.reshape(1, -1)  if it contains a single sample.
#normalizer.fit(X_train['price'].values.reshape(-1,1))
normalizer.fit(X_train['price'].values.reshape(1,-1))

X_train_price_norm = normalizer.transform(X_train['price'].values.reshape(1,-1))
X_cv_price_norm = normalizer.transform(X_cv['price'].values.reshape(1,-1))
X_test_price_norm = normalizer.transform(X_test['price'].values.reshape(1,-1))

X_train_price_norm =X_train_price_norm.reshape(-1,1)
X_cv_price_norm = X_cv_price_norm.reshape(-1,1)
X_test_price_norm = X_test_price_norm.reshape(-1,1)

print("After vectorizations")
#print(X_train_price_norm_1.shape, y_train.shape)
print(X_train_price_norm.shape, y_train.shape)
print(X_cv_price_norm.shape, y_cv.shape)
print(X_test_price_norm.shape, y_test.shape)
print("="*100)


## 1.4.7 encoding numerical features: teacher_number_of_previously_posted_projects

In [ ]:

from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
# normalizer.fit(X_train['price'].values)
# this will rise an error Expected 2D array, got 1D array instead: 
# array=[105.22 215.96  96.01 ... 368.98  80.53 709.67].
# Reshape your data either using 
# array.reshape(-1, 1) if your data has a single feature 
# array.reshape(1, -1)  if it contains a single sample.
normalizer.fit(X_train['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))

X_train_teacher_number_of_previously_posted_projects_norm = normalizer.transform(X_train['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))
X_cv_teacher_number_of_previously_posted_projects_norm = normalizer.transform(X_cv['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))
X_test_teacher_number_of_previously_posted_projects_norm = normalizer.transform(X_test['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))

X_train_teacher_number_of_previously_posted_projects_norm = X_train_teacher_number_of_previously_posted_projects_norm.reshape(-1,1)
X_cv_teacher_number_of_previously_posted_projects_norm    = X_cv_teacher_number_of_previously_posted_projects_norm.reshape(-1,1)
X_test_teacher_number_of_previously_posted_projects_norm  = X_test_teacher_number_of_previously_posted_projects_norm.reshape(-1,1)

print("After vectorizations")
print(X_train_teacher_number_of_previously_posted_projects_norm.shape, y_train.shape)
print(X_cv_teacher_number_of_previously_posted_projects_norm.shape, y_cv.shape)
print(X_test_teacher_number_of_previously_posted_projects_norm.shape, y_test.shape)
print("="*100)
#X_train_teacher_number_of_previously_posted_projects_norm

## 1.4.8 Score of Eassay

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import nltk
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

def get_essay_fea_score(feature, df):
   
    #value_count = X_train[feature].value_counts()
    vec =[]
   
    for index, row in df.iterrows():
      X_train_essay_score = sid.polarity_scores(row[feature])
      vec.append(list(X_train_essay_score.values()))
      
    return vec

In [ ]:
print(X_train.shape, y_train.shape)
print(X_cv.shape, y_cv.shape)
print(X_test.shape, y_test.shape)

print("="*100)


X_train_essay_feature_score = np.array(get_essay_fea_score("essay", X_train))
X_cv_essay_feature_score = np.array(get_essay_fea_score("essay", X_cv))
X_test_essay_feature_score = np.array(get_essay_fea_score("essay", X_test))

#X_train_essay_feature_score = X_train_essay_feature_score.reshape(-1,1)
#X_cv_essay_feature_score = X_cv_essay_feature_score.reshape(-1,1)
#X_test_essay_feature_score = X_test_essay_feature_score.reshape(-1,1)

print("After vectorizations")
print(X_train_essay_feature_score.shape, y_train.shape)
print(X_cv_essay_feature_score.shape, y_cv.shape)
print(X_test_essay_feature_score.shape, y_test.shape)
print("="*100)

## Concatinating all the features

In [ ]:

# merge two sparse matrices: https://stackoverflow.com/a/19710648/4084039
from scipy.sparse import hstack
X_tr = hstack((X_train_essay_tfidf, X_train_state_feature_responseCoding, X_train_teacher_prefix_feature_responseCoding, X_train_project_grade_category_feature_responseCoding,X_train_clean_categories_feature_responseCoding,X_train_clean_subcategories_feature_responseCoding, X_train_price_norm,X_train_teacher_number_of_previously_posted_projects_norm,X_train_essay_feature_score)).tocsr()
X_cr = hstack((X_cv_essay_tfidf, X_cv_state_feature_responseCoding, X_cv_teacher_prefix_feature_responseCoding, X_cv_project_grade_category_feature_responseCoding, X_cv_clean_categories_feature_responseCoding,X_cv_clean_subcategories_feature_responseCoding,X_cv_price_norm,X_cv_teacher_number_of_previously_posted_projects_norm,X_cv_essay_feature_score)).tocsr()
X_te = hstack((X_test_essay_tfidf, X_test_state_feature_responseCoding, X_test_teacher_prefix_feature_responseCoding, X_test_project_grade_category_feature_responseCoding,X_test_clean_categories_feature_responseCoding,X_test_clean_subcategories_feature_responseCoding, X_test_price_norm,X_test_teacher_number_of_previously_posted_projects_norm,X_test_essay_feature_score)).tocsr()

print("Final Data matrix")
print(X_tr.shape, y_train.shape)
print(X_cr.shape, y_cv.shape)
print(X_te.shape, y_test.shape)
print("="*100)

<h2>1.5 Appling Models on different kind of featurization as mentioned in the instructions</h2>

<br>Apply GBDT on different kind of featurization as mentioned in the instructions
<br> For Every model that you work on make sure you do the step 2 and step 3 of instrucations

In [ ]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

1.5.1 Appling DT: TFIDF featurization

1.5.1.1 Hyper parameter Tuning

1.5.1.1.2 - Random Search method

In [ ]:

from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier


#k =[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000,10000]
K=[1, 5, 10, 50]
alpha = [100,200,500,1000,2000]
#min_samples_split= [5, 10, 100, 500]
#C = uniform(0.00001,10000)
#parameters = {'alpha':uniform(0.00001,10000)}
parameters = dict(max_depth=K,n_estimators=alpha)
clf = RandomizedSearchCV(GradientBoostingClassifier(), parameters, cv=3, scoring='roc_auc',return_train_score=True)
clf.fit(X_tr, y_train)

results = pd.DataFrame.from_dict(clf.cv_results_)
#results = results.sort_values(['param_alpha'])
results = results.sort_values(['param_max_depth'])
train_auc= results['mean_train_score']
#train_auc_std= results['std_train_score']
cv_auc = results['mean_test_score'] 
#cv_auc_std= results['std_test_score']
depth =  results['param_max_depth']
estimators = results['param_n_estimators']
results.head()

In [ ]:
print(clf.best_params_)

 # AUC Matrix with Train data

In [ ]:
from pandas import DataFrame
import seaborn as sns
%matplotlib inline

df1 = results[['mean_train_score','param_max_depth','param_n_estimators']]
#df1.head()
heatmap1_data = pd.pivot_table(df1, values='mean_train_score', 
                     index=['param_n_estimators'], 
                     columns='param_max_depth')
sns.heatmap(heatmap1_data, annot=True)


# AUC Matrix with Test data

In [ ]:

df2 = results[['mean_test_score','param_max_depth','param_n_estimators']]
heatmap2_data = pd.pivot_table(df2, values='mean_test_score', 
                     index=['param_n_estimators'], 
                     columns='param_max_depth')
sns.heatmap(heatmap2_data, annot=True)


1.5.1.2 Testing the performance of the model on test data, plotting ROC Curves

In [ ]:
# from the error plot we choose K such that, we will have maximum AUC on cv data and gap between the train and cv is less
# Note: based on the method you use you might get different hyperparameter values as best one
# so, you choose according to the method you choose, you use gridsearch if you are having more computing power and note it will take more time
# if you increase the cv values in the GridSearchCV you will get more rebust results.

#here we are choosing the best_k based on forloop results
best_depth = 1
best_estimator = 1000

In [ ]:

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import GradientBoostingClassifier


clf = GradientBoostingClassifier(max_depth=best_depth,n_estimators=best_estimator)
clf.fit(X_tr, y_train)
# roc_auc_score(y_true, y_score) the 2nd parameter should be probability estimates of the positive class
# not the predicted outputs

#y_train_pred = batch_predict(clf, X_tr)
y_train_pred = clf.predict(X_tr)    
#y_test_pred = batch_predict(clf, X_te)
y_test_pred = clf.predict(X_te)

train_fpr, train_tpr, tr_thresholds = roc_curve(y_train, y_train_pred)
test_fpr, test_tpr, te_thresholds = roc_curve(y_test, y_test_pred)

plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
plt.legend()
plt.xlabel("K: hyperparameter")
plt.ylabel("AUC")
plt.title("ERROR PLOTS")
plt.grid()
plt.show()

In [ ]:
# we are writing our own function for predict, with defined thresould
# we will pick a threshold that will give the least fpr
def find_best_threshold(threshould, fpr, tpr):
    t = threshould[np.argmax(tpr*(1-fpr))]
    # (tpr*(1-fpr)) will be maximum if your fpr is very low and tpr is very high
    print("the maximum value of tpr*(1-fpr)", max(tpr*(1-fpr)), "for threshold", np.round(t,3))
    return t

def predict_with_best_t(proba, threshould):
    predictions = []
    for i in proba:
        if i>=threshould:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions

In [ ]:
print("="*100)
from sklearn.metrics import confusion_matrix
best_t = find_best_threshold(tr_thresholds, train_fpr, train_tpr)
print("Train confusion matrix")
print(confusion_matrix(y_train, predict_with_best_t(y_train_pred, best_t)))
print("Test confusion matrix")
print(confusion_matrix(y_test, predict_with_best_t(y_test_pred, best_t)))

# Train Confusion Matrix

In [ ]:

#https://stackoverflow.com/questions/35572000/how-can-i-plot-a-confusion-matrix
import seaborn as sns
import matplotlib.pyplot as plt
ax= plt.subplot()
sns.heatmap(confusion_matrix(y_train, predict_with_best_t(y_train_pred, best_t)), annot=True, ax = ax,fmt='g');

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

## Test Confusion Matrix

In [ ]:
#https://stackoverflow.com/questions/35572000/how-can-i-plot-a-confusion-matrix
import seaborn as sns
import matplotlib.pyplot as plt
ax= plt.subplot()
sns.heatmap(confusion_matrix(y_test, predict_with_best_t(y_test_pred, best_t)), annot=True, ax = ax,fmt='g');

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');

1.5.2 Appling Decision Tree: TFIDFW2V

In [ ]:
# stronging variables into pickle files python: http://www.jessicayung.com/how-to-use-pickle-to-save-and-load-variables-in-python/
# make sure you have the glove_vectors file
with open('C:/Users/Abhishek V/Downloads/glove_vectors (3)', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

In [ ]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
tfidf_model = TfidfVectorizer()
tfidf_model.fit(X_train['essay'].values)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
tfidf_words = set(tfidf_model.get_feature_names())

In [ ]:
def tfidf_w2v(df,feature):
  
  essay_tfidf_w2v_vectors = []; # the avg-w2v for each sentence/review is stored in this list
  for sentence in tqdm(df[feature].values): # for each review/sentence
      vector = np.zeros(300) # as word vectors are of zero length
      tf_idf_weight =0; # num of words with a valid vector in the sentence/review
      for word in sentence.split(): # for each word in a review/sentence
          if (word in glove_words) and (word in tfidf_words):
              vec = model[word] # getting the vector for each word
              # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
              tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
              vector += (vec * tf_idf) # calculating tfidf weighted w2v
              tf_idf_weight += tf_idf
      if tf_idf_weight != 0:
          vector /= tf_idf_weight
      essay_tfidf_w2v_vectors.append(vector)
      
  return essay_tfidf_w2v_vectors

#X_train_essay_tfidf_w2v_vectors_2 = np.array(tfidf_w2v(X_train,'essay'))

In [ ]:
X_train_essay_tfidf_w2v_vectors = np.array(tfidf_w2v(X_train,'essay'))
X_cv_essay_tfidf_w2v_vectors    = np.array(tfidf_w2v(X_cv,'essay'))
X_test_essay_tfidf_w2v_vectors    = np.array(tfidf_w2v(X_test,'essay'))

In [ ]:
print(X_train.shape, y_train.shape)
print(X_cv.shape, y_cv.shape)
print(X_test.shape, y_test.shape)

print("="*100)
print("After vectorizations")
print(X_train_essay_tfidf_w2v_vectors.shape, y_train.shape)
print(X_cv_essay_tfidf_w2v_vectors.shape, y_cv.shape)
print(X_test_essay_tfidf_w2v_vectors.shape, y_test.shape)
print("="*100)


1.5.2.2 Concatinating all the features

In [ ]:
# merge two sparse matrices: https://stackoverflow.com/a/19710648/4084039
from scipy.sparse import hstack
from scipy import sparse
X_train_essay_tfidf_w2v_vectors = sparse.csr_matrix(X_train_essay_tfidf_w2v_vectors)
X_cv_essay_tfidf_w2v_vectors = sparse.csr_matrix(X_cv_essay_tfidf_w2v_vectors)
X_test_essay_tfidf_w2v_vectors = sparse.csr_matrix(X_test_essay_tfidf_w2v_vectors)
X_tr = hstack((X_train_essay_tfidf_w2v_vectors, X_train_state_feature_responseCoding, X_train_teacher_prefix_feature_responseCoding, X_train_project_grade_category_feature_responseCoding,X_train_clean_categories_feature_responseCoding,X_train_clean_subcategories_feature_responseCoding, X_train_price_norm,X_train_teacher_number_of_previously_posted_projects_norm,X_train_essay_feature_score)).tocsr()
X_cr = hstack((X_cv_essay_tfidf_w2v_vectors, X_cv_state_feature_responseCoding, X_cv_teacher_prefix_feature_responseCoding, X_cv_project_grade_category_feature_responseCoding, X_cv_clean_categories_feature_responseCoding,X_cv_clean_subcategories_feature_responseCoding,X_cv_price_norm,X_cv_teacher_number_of_previously_posted_projects_norm,X_cv_essay_feature_score)).tocsr()
X_te = hstack((X_test_essay_tfidf_w2v_vectors, X_test_state_feature_responseCoding, X_test_teacher_prefix_feature_responseCoding, X_test_project_grade_category_feature_responseCoding,X_test_clean_categories_feature_responseCoding,X_test_clean_subcategories_feature_responseCoding, X_test_price_norm,X_test_teacher_number_of_previously_posted_projects_norm,X_test_essay_feature_score)).tocsr()
print("Final Data matrix")
print(X_tr.shape, y_train.shape)
print(X_cr.shape, y_cv.shape)
print(X_te.shape, y_test.shape)
print("="*100)


1.5.3.1 Hyper parameter Tuning

1.5.3.1.1 Method 1: Simple for loop (if you are having memory limitations use this)

1.5.3.1.2 Method 2:-RandomSearch

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

K=[1, 5, 10, 50]
alpha = [100,200,500,1000,2000]
#min_samples_split= [5, 10, 100, 500]
#C = uniform(0.00001,10000)
#parameters = {'alpha':uniform(0.00001,10000)}
parameters = dict(max_depth=K,n_estimators=alpha)

clf = RandomizedSearchCV(GradientBoostingClassifier(), parameters, cv=3, scoring='roc_auc',return_train_score=True)
clf.fit(X_tr, y_train)

results = pd.DataFrame.from_dict(clf.cv_results_)
#results = results.sort_values(['param_alpha'])
results = results.sort_values(['param_max_depth'])
train_auc= results['mean_train_score']
#train_auc_std= results['std_train_score']
cv_auc = results['mean_test_score'] 
#cv_auc_std= results['std_test_score']
depth =  results['param_max_depth']
estimators = results['param_n_estimators']
results.head()

In [ ]:

print(clf.best_params_)

# AUC Matrix with Train data

In [ ]:
from pandas import DataFrame
import seaborn as sns
%matplotlib inline

df1 = results[['mean_train_score','param_max_depth','param_n_estimators']]
#df1.head()
heatmap1_data = pd.pivot_table(df1, values='mean_train_score', 
                     index=['param_n_estimators'], 
                     columns='param_max_depth')
sns.heatmap(heatmap1_data, annot=True)

In [ ]:

AUC Matrix with Test data

In [ ]:
df2 = results[['mean_test_score','param_max_depth','param_n_estimators']]
heatmap2_data = pd.pivot_table(df2, values='mean_test_score', 
                     index=['param_n_estimators'], 
                     columns='param_max_depth')
sns.heatmap(heatmap2_data, annot=True)


1.5.3.2 Testing the performance of the model on test data, plotting ROC Curves

In [ ]:
best_depth =1
best_estimator = 500

In [ ]:

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import GradientBoostingClassifier


clf = GradientBoostingClassifier(max_depth=best_depth,n_estimators=best_estimator)
clf.fit(X_tr, y_train)
# roc_auc_score(y_true, y_score) the 2nd parameter should be probability estimates of the positive class
# not the predicted outputs

#y_train_pred = batch_predict(clf, X_tr)
y_train_pred = clf.predict(X_tr)    
#y_test_pred = batch_predict(clf, X_te)
y_test_pred = clf.predict(X_te)

train_fpr, train_tpr, tr_thresholds = roc_curve(y_train, y_train_pred)
test_fpr, test_tpr, te_thresholds = roc_curve(y_test, y_test_pred)

plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
plt.legend()
plt.xlabel("K: hyperparameter")
plt.ylabel("AUC")
plt.title("ERROR PLOTS")
plt.grid()
plt.show()


## 3.Representation of results

In [ ]:
print("="*100)
from sklearn.metrics import confusion_matrix
best_t = find_best_threshold(tr_thresholds, train_fpr, train_tpr)
print("Train confusion matrix")
print(confusion_matrix(y_train, predict_with_best_t(y_train_pred, best_t)))
print("Test confusion matrix")
print(confusion_matrix(y_test, predict_with_best_t(y_test_pred, best_t)))

In [ ]:
#https://stackoverflow.com/questions/35572000/how-can-i-plot-a-confusion-matrix
import seaborn as sns
import matplotlib.pyplot as plt
ax= plt.subplot()
sns.heatmap(confusion_matrix(y_train, predict_with_best_t(y_train_pred, best_t)), annot=True, ax = ax,fmt='g');

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');



## Test Confusion Matrix

In [ ]:

#https://stackoverflow.com/questions/35572000/how-can-i-plot-a-confusion-matrix
import seaborn as sns
import matplotlib.pyplot as plt
ax= plt.subplot()
sns.heatmap(confusion_matrix(y_test, predict_with_best_t(y_test_pred, best_t)), annot=True, ax = ax,fmt='g');

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');


# 4. Summary

In [ ]:

from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Vectorizer","Model","HYPER PARAMETER-depth , n_estimators","AUC"]

x.add_row(["TFIDF","GBDT-RandomSearch","1,1000",0.5103])
x.add_row(["TFIDF W2V","GBDT-RandomSearch","1,500",0.504])


print(x)